In [7]:
!pip install flask-ngrok

In [8]:
!pip install transformers

In [9]:
!pip install flask pyngrok

In [10]:
!ngrok config add-authtoken 2ql2kppXAVm6ve9hZmYfeM6AzuD_2LgQfU186nM7NL1KpDVRn

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [11]:
from pyngrok import ngrok

public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

Public URL: NgrokTunnel: "https://1e82-35-230-163-92.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
from flask import Flask, request, jsonify
from transformers import AutoModelForCausalLM, AutoTokenizer


class Dialog:
    def __init__(self):
        # モデルとトークナイザーのロード
        # model_name = "rinna/japanese-gpt-neox-3.6b"
        model_name = "rinna/japanese-gpt-1b"
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        self.history = []

    def resp_echo(self, user_input_text):
        return f"{user_input_text}"

    def generate_response(self, user_input):
        # 会話履歴を更新
        self.history.append(f"ユーザー: {user_input}")
        input_text = "\n".join(self.history)

        # 応答生成
        inputs = self.tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
        output = self.model.generate(
            inputs,
            max_length=512,
            num_return_sequences=1,
            pad_token_id=self.tokenizer.eos_token_id,
            no_repeat_ngram_size=2,
            top_p=0.9,
            temperature=0.8
        )

        # 応答をデコード
        response = self.tokenizer.decode(output[0], skip_special_tokens=True)
        response = response.split("モデル:")[-1].strip()  # 応答部分を抽出

        # 会話履歴を更新
        self.history.append(f"モデル: {response}")
        return response

dialog_model = Dialog()

app = Flask(__name__)

@app.route('/process', methods=['POST'])
def process_request():
    data = request.json
    user_input = data.get("text")
    if not user_input:
        return jsonify({"error": "Invalid input"}), 400

    response = dialog_model.generate_response(user_input)
    result = {"response": response}
    return jsonify(result)


if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [31/Dec/2024 17:30:08] "POST / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [31/Dec/2024 17:30:26] "POST / HTTP/1.1" 404 -
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` o